Before we start: please fill out our weekly attendance form! https://forms.gle/njrwXR9r416yzXnn7

Fall quarter outline:

* Week 3: Introduction
* Week 4: Data retrieval and preparation
* Week 5: Exploratory data analysis (EDA)
* Week 6: Modeling and machine learning, part 1
* **Week 7: Modeling and machine learning, part 2**
    * Recap of last week
    * Supervised learning, continued
        * Classification algorithms
        * Regression algorithms
* *Week 8: Thanksgiving*
* Week 9: Neural networks 

<img src="https://miro.medium.com/max/1200/1*eE8DP4biqtaIK3aIy1S2zA.png" width="800">

### Mounting Google Drive

Don't worry about this; this bit of code is only necessary because we're working in Google Colab instead of Jupyter Notebook. I'm mounting Google Drive to this notebook so that I can access files that are stored there.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd "drive/My Drive/DSU 2020-2021/Curriculum/Datasets"

In [ ]:
!ls

# Recap

## Overview

To recap last week, **machine learning** is done through machine learning **algorithms**, all of which use **training data** to improve their performance on a specific task.

<!-- Per the editor of *Introducing Data Science*, "Machine learning is the process by which a computer can work more accurately as it collects and learns from the data it is given." -->

The two main types of machine learning are **unsupervised** learning and **supervised** learning. 
* *Unsupervised* &rarr; *unlabeled* training data
* *Supervised* &rarr; *labeled* training data

In general, most of our problems will be supervised.

Supervised learning can be further broken down into **two categories**:
* **Regression** &rarr; continuous (numeric) target variable
* **Classification** &rarr; discrete (categorical) target variable


Last week, we went through two examples of unsupervised algorithms: PCA (for signal separation), and K-means (for clustering).

This week, we're going to focus on several **supervised learning algorithms**, namely:
* For **classification**: KNN, naive Bayes, decision trees, logistic regression
* For **regression**: (KNN, decision trees,) linear models, neural networks

However, keep in mind that **we are not covering every single ML algorithm!** There are wayyy more algorithms than we have time to touch on, and different models perform better on different data. You can find a pretty comprehensive list at the [scikit-learn user guide](https://scikit-learn.org/stable/user_guide.html).

## The scikit-learn package

Speaking of scikit-learn, let's recap how to train <strike>your dragon</strike> a supervised model using it.

Once you've decided which algorithm to use, you import it from the correct `sklearn` module and initialize it using its constructor, specifying optional **hyperparameters** &mdash; parameters whose value control the training process.

```
from sklearn.[module] import [Estimator]

model = Estimator([optional hyperparameters])
```

<span style="font-size:8pt;">
(You should read the documentation as needed to understand each model's parameters and their default values.)
</span>

After initializing the model, all you have to do to train the model is call the `.fit()` method with your training data:

```
model.fit(X)
```

<span style="font-size:8pt;">
(Note: Per the <a href="https://scikit-learn.org/stable/faq.html">sklearn FAQ</a>, <code>X</code> must be numeric data stored as a <code>numpy</code> array. This includes <code>pandas.DataFrame</code> since they're convertible to <code>numpy</code> arrays.)
</span>

Then, for supervised models, you call the `.predict()` method to generate predictions from the training data (or new data):

```
predictions = model.predict(X)
```

And that's it! Each model will also have a number of additional methods and attributes that you can access, but again, you'll have to refer to the documentation as you go.

## Fundamental concepts

Mathematically speaking, **the goal of supervised learning** is to take a set of *training observations* $\{x_1, x_2, \dots, x_n\}$ and their *target* labels $\{t_1, t_2, \dots, t_n\}$ and **learn a function** $y(x)$ such that $y(x_i) \approx t_i$ for all the training data.

### Train-test split

To see how well our model **generalizes** to new data, we can use **train-test split**, in which we randomly split our data into *training* and *testing* subsets (usually in a 80/20 or 70/30 ratio). The model *learns* from the training set, and to estimate its *generalization error*, we make predictions on the test set and compare them with the actual targets.

*New this week*: 

I'll demonstrate how to split data using the `train_test_split()` function from `sklearn.model_selection`. But, it literally takes one line of code:

```
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = __)
```

First, importing the `iris` dataset:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

X

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8) # 80/20 split

print("X.shape:      ", X.shape)
print("X_train.shape:", X_train.shape)
print("X_test.shape: ", X_test.shape)
print("")
print("y.shape:      ", y.shape)
print("y_train.shape:", y_train.shape)
print("y_test.shape: ", y_test.shape)
print("")

X_train

### Overfitting and underfitting, bias-variance tradeoff

<img src="https://www.educative.io/api/edpresso/shot/6668977167138816/image/5033807687188480" width = "700px">

The purpose of the train-test split is to see if our model **overfits** the data, **underfits** the data, or fits the data well. In general:

* *Underfitting* &rarr; make the model *more complex* (more features/parameters)
* *Overfitting* &rarr; make the model *simpler* (fewer features/parameters; regularization)


<img src="https://www.learnopencv.com/wp-content/uploads/2017/02/Bias-Variance-Tradeoff-In-Machine-Learning-1.png" width="600px">

To understand overfitting/underfitting better, we introduced the idea of **bias-variance tradeoff**. Basically, no matter what ML algorithm we choose, we will always need to find a balance between its *bias* and its *variance*.

* Model **bias** is *training error* (MSE for regression, or misclassification rate for classification). High bias &rarr; underfitting.
* Model **variance** is how *sensitive* a model is to *noise* in the training data. High variance &rarr; overfitting.

### Cross-validation

To select the optimal hyperparameters for our model, we can use **cross-validation**. Cross-validation is also a more robust strategy to determine how well our model generalizes.

To perform cross-validation, we divide our data into **three groups**: training, validation, and testing and then follow these steps:

* Use the *training set* to actually train the model using different combinations of *hyperparameters*. 
* Calculate the model's *validation error* using the *validation set*. Typically, we'll choose the combination of hyperparameters which minimizes the validation error.
* Estimate the model's generalization error using the *test set*.

The technique we usually use is **K-fold cross-validation**:
* Randomly divide the training observations into $K$ groups (or folds). 
* For each iteration, one group is selected as the validation set, while the $K-1$ others are used as the training set.

<!-- * **Leave-one-out cross-validation (LOOCV)**: the same as K-fold, except we take $K=N$ (# of training observations). For each iteration, just one observation is selected as the validation set, and we repeat over the entire training set. -->

There's also **leave-one-out cross-validation (LOOCV)** in which we take $K=N$ (the number of training observations), but this is very computationally expensive on large datasets.

*New this week:*

I'll demonstrate how to perform K-fold cross validation using functions from `sklearn.model_selection`. But we can't do that without a model first, so I'll run it at the end of the first example below (KNN).

# Classification algorithms

## K Nearest Neighbors (KNN)

With the **K-nearest neighbors (KNN) classifier**, we classify a point based on the classes of the $K$ points closest in distance to it. 

Distance here is usually the Euclidean distance: $d = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2}$.

<!-- <img src="https://www.oreilly.com/library/view/hands-on-recommendation-systems/9781788993753/assets/1c808a35-3c9d-4bbe-a6ae-e858a3961159.png" width = "800px"> -->

We'll focus on a 2D example, but we could use more than two features and the algorithm would factor this into the distance calculations, e.g. the distance between 2 points in 3D is $d = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2 + (z_2-z_1)^2}$.

The algorithm goes as follows:


1.   Calculate the distance between the point we wish to classify and all other points in the dataset. (KNN doesn't require splitting the data into training/testing sets.)
2.   Order these distances.
3.   Count how many of the *K* nearest neighbors belong to each class. (We tell the algorithm what number to use for *K*.)
4.   Whichever class is the most common among the *K* nearest neighbors is the class we assign the new point to.

<img src = "https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1531424125/KNN_final1_ibdm8a.png" height = "450px" width = "600px">

You can see that how we classify the new point depends on what value we choose for *K*.



### Coding KNN

In [ ]:
# load the iris data set and pandas library
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)

df.head()

In [ ]:
# 3 classes (0 = "setosa", 1 = "versicolor", 2 = "virginica")
iris.target, iris.target_names

In [ ]:
# for simplicity's sake we'll only use sepal length and sepal width
X = df[['petal length (cm)', 'sepal width (cm)']]
y = iris.target

# scale the data
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# load and set up the KNN classifier w/ K=3
from sklearn.neighbors import KNeighborsClassifier
knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(X_scaled, y)

# test it out with sepal length = 0.5 and sepal width = 2.6
prediction = knn3.predict([[0.5, 2.6]])
print(iris.target_names[prediction])

In [ ]:
# try the same but with K=6
knn6 = KNeighborsClassifier(n_neighbors=6)
knn6.fit(X_scaled, y)
prediction = knn6.predict([[0.5, 2.6]])
print(iris.target_names[prediction])

In [ ]:
# how did we do?
print("K=3, accuracy:", knn3.score(X_scaled, y))
print("K=6, accuracy:", knn6.score(X_scaled, y))

In [ ]:
from sklearn.metrics import confusion_matrix

print("K=3, confusion matrix:")
print(confusion_matrix(y, knn3.predict(X_scaled)))
print("")
print("K=6, confusion matrix:")
print(confusion_matrix(y, knn6.predict(X_scaled)))

### How do we choose *k*?

*   Higher values of *k* require more computation, while lower values of k make the algorithm more error-prone.
*   We typically choose *k* to be an odd number to avoid ties between classes.
*   The rule of thumb is if we have *n* observations, then *k* should be $\sqrt{n}$.

**Quiz time!**

<img src = "https://miro.medium.com/max/754/1*VZCDAcvMqlU3bPlb5BfPug.png" width = "400px">

**Which class do we assign the new point (the red one) if we choose k=3?**  
**A**. Class A  
**B**. Class B    
**How about if k=6?**

### Cross validation, cont.

Another way to choose the $K$ for KNN is through K-fold cross-validation! To do that in `sklearn`, we can use the `cross_val_score()` function from `sklearn.model_selection`:

In [ ]:
from sklearn.model_selection import cross_val_score

mean_fold_errors = []

for num_neighbors in range(1, 20):
    np.random.seed(0)   # to ensure we use the same random folds

    knn = KNeighborsClassifier(num_neighbors)
    fold_errors = 1 - cross_val_score(knn, X_scaled, y, cv = 5)   # 5-fold CV
    mean_fold_errors.append(np.mean(fold_errors))

    print("")
    print("Num neighbors =", num_neighbors)
    print("K-fold CV errors:", np.round(fold_errors, 3))
    print("K-fold CV mean error:", np.round(mean_fold_errors[-1], 3))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, 20), mean_fold_errors)
plt.xticks(range(1, 20, 2))
plt.xlabel('$K$ (Num Neighbors)')
plt.ylabel('Mean CV Error')
plt.show()

## Naive Bayes

The **Naive Bayes classifier** requires knowing a bit about Bayes Rule:

<img src = "https://miro.medium.com/max/1994/1*CnoTGGO7XeUpUMeXDrIfvA.png" height = "400px" width = "500 px">

Bayes’ Rule is a law of probability that describes the relationship between the probability of an event occurring, given a certain piece of information (**the “posterior”**), and the probability of the event occurring without knowledge of this information (**the “prior”**).

**Terminology check:** if **probability** is the chance of observing some data given some parameter assumption (e.g. if mean = 0 and sd = 1, what are the chances we observe 0.5?), then **likelihood** is the chance that the parameter is some value given that we've observed some data (e.g. if we observe 0.5, what are the chances that mean = 0 and sd = 1?).

Back to the Bayes Rule formula. In Naive Bayes we can ignore the denominator P(B) because it's a constant and all that matters is that the Posterior is **proportional** to  Likelihood * Prior. This makes for easier calculations!

The classifier is called "naive" because we assume a) our observations are independent, and b) our features are normally distributed. This is called **Gaussian Naive Bayes**; there are other variations of this algorithm, but we'll focus on the Gaussian case here.

In the context of the Iris data set, assume we're looking for the probability that a new obsevation is a setosa. Bayes Rule then looks like: $$P(X = \text{setosa} \mid \text{some observed data } \mu \text { and } \sigma) \propto P(\text{observing } \mu \text { and } \sigma \mid X = \text{setosa}) * P(\text{any observation is setosa})$$

As you might have guessed, Naive Bayes returns a probability that an observation returns to a class, so even though it's a categorical classifier, it's still probabilistic.

### Coding Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

X = df
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# train the model
nb = GaussianNB()
nb.fit(X_train, y_train)

In [ ]:
# we don't actually have to set the prior probabilities -- GaussianNB() adjusts them to the data
nb.class_prior_

In [ ]:
# test the model
y_predicted = nb.predict(X_test)
print(y_predicted)
print("")

# the underlying list of probabilities
print(np.round(nb.predict_proba(X_test), 2))

In [ ]:
# how did we do?
nb.score(X_test, y_test)

When should we use Naive Bayes?

**Pros:**
*   Super quick
*   Relatively simple
*   Doesn't require a ton of data to make a decent clasfication

**Cons:**
*    Have to assume data follows a distribution
*    Have to assume independence
*    Only performs well when classes are very separated

## Decision trees

**Decision trees** are another method we can use to classify data. They're best explained by providing a simple example:

<img src = "https://d2h0cx97tjks2p.cloudfront.net/blogs/wp-content/uploads/sites/2/2017/07/Decision-Trees-Example.png" width = "450 px">

We see that a decision tree consists of several *nodes* connected by *branches*. At each node, we make a "decision" based off a certain criterion (e.g. "Is it raining?"), and we follow these nodes and branches until we reach a *terminal node*. To classify an observation, we start at the top and use the criteria contained in each node to decide which branch to follow.

<span style="font-size:8pt;">
[Mathematically, the algorithm determines which criterion is "best" at each node by minimizing the <i>impurity</i> resulting from all possible criteria (each category for categorical predictors, thresholds for numeric predictors). The most common impurity measure is <i>Gini impurity</i>: 
<br>
$\sum\limits_k p_{k} (1 - p_{k}) = 1 - \sum\limits_k p_{k}^2$
<br>
where each $k$ represents one of the target classes of the data, and $p_k$ is the proportion of the data belonging to that class.]
</span>


Decision trees have several pros and cons.

**Pros:**
* They are simple and easy to understand
* It's very quick to classify new observations
* They can handle numeric and categorical predictors

**Cons:**
* Computationally expensive to train for large datasets
* They are often inaccurate
* They are prone to overfitting

However, we can address two of these disadvantages by using an **ensemble method** called **Random Forest**, which trains many decision trees using random splits of the data and averaging the results.

### Coding Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

X = df
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# train the model
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

Another advantage of decision trees is that we can actually plot the model itself:

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize=(10,8))
plot_tree(dt, feature_names=iris.feature_names, class_names=iris.target_names,
          filled=True, fontsize=9)
plt.show()

In [ ]:
# how did we do?
dt.score(X_test, y_test)

## Logistic Regression

**Logistic regression**, despite its name, is a linear model for classification. It works by assuming that you can approximate the probability that an observation belongs to a certain class with the **sigmoid (or logistic) function**: $\sigma(x) = \dfrac{1}{1+e^{-x}}$.

In [ ]:
x = np.linspace(-8, 8)
y = 1 / (1 + np.exp(-x))

plt.plot(x, y)
plt.grid(True)
plt.xlabel('x')
plt.ylabel('$\sigma(x)$')
plt.show()

To classify an observation $X = (x_1, \dots, x_n)$, we plug in a linear combination of the predictors into the sigmoid function: $y(x) = \sigma(b_0 + b_1 x_1 + \dots + b_n x_n)$, and take the class for which the predicted probability is greatest.

<span style="font-size:8pt;">
[Mathematically, the algorithm works by minimizing the <i>cross-entropy</i>. In the binary case, this is: 
<br>
$-\sum\limits_{i=1}^{N} \{t_n \ln y(x_n) + (1-t_n) \ln (1-y(x_n))\}$
<br>
since the predicted probabilities between the two classes sum to 1. The model uses <i>gradient descent</i> to solve for the optimal solution for the coefficients.]
</span>


### Coding Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

X = df
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# train the model
lr = LogisticRegression(max_iter=200)   # (I only specify max_iter b/c the default max isn't enough here)
lr.fit(X_train, y_train)

In [ ]:
# the model's coefficients (b0, b1, ..., bn)
# note that there are three sets of coeffs, since there are three classes
lr.intercept_, lr.coef_

In [ ]:
# the predicted probabilities of each class
np.round(lr.predict_proba(X_train), 2)

In [ ]:
# how did we do?
lr.score(X_test, y_test)

# Regression algorithms

## KNN and decision trees

These two algorithms again? Yup! 

With some small modifications, each of these classification algorithms (and some others) can also be used for regression. We won't explain them in much more detail, but here's a code example of each one:

In [ ]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = diabetes.target # a measure of disease progression one year after baseline

X

In [ ]:
y

As with classification, we want to scale our data before applying KNN. Aside from calling `StandardScaler()` and KNN separately, we can also combine them using an `sklearn` `Pipeline`:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# fit the model
knr = make_pipeline(StandardScaler(), KNeighborsRegressor())
knr.fit(X_train, y_train)

In [ ]:
knr.score(X_test, y_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)

In [ ]:
plot_tree(dtr) # this might take a while...
plt.show()

In [ ]:
dtr.score(X_test, y_test) # the model's R^2, not good

## Linear regression

The goal of **linear regression** is to model the relationship between predictor variables (inputs) and a response variable (output). The equation we use to model this relationship follows the form:

<img src= "https://csharpcorner-mindcrackerinc.netdna-ssl.com/article/linear-regression2/Images/f_MLR.png" width = "500px">


How do we find this equation? In other words, how do we find the coefficients $\beta_0,...,\beta_n?$ The goal is to minimize our model's error, or **loss**. A common loss function in linear regression is

$$L = \sum_{i=1}^{n} (y_i-\hat{y_i})^2$$ where we add up the squares of the **residuals** or the differences between our observed values ($y_i$) and the values predicted by our regression equation ($\hat{y_i}$).

We want to minimize this loss function. A little calculus can get us the answer, but since we won't ever actually have to calculate the regression equation by hand, we don't need to do the calculus here. Just remember that **the "best" linear model is the one that minimizes the loss function.**

### How do we measure the strength of a linear model?

Straightforward: take the average of the loss function from earlier. We call this the **mean squared error (MSE)**.

$$MSE = \frac{1}{n}\sum_{i=1}^{n} (y_i-\hat{y_i})^2$$

In general, the lower the MSE the stronger the model. (But if it's too low we might be overfitting!)

### Coding linear regression

In [ ]:
# once again we load the appropriate dependencies from scikit learn
# we will also load scikit learn's diabetes dataset
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()

X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = diabetes.target # a measure of disease progression one year after baseline

In [ ]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# fit the model
lm = LinearRegression()
lm.fit(X_train, y_train)

# let's see what coefficients the model chose
lm.coef_

In [ ]:
# and the intercept
lm.intercept_

In [ ]:
# let's see what the model predicts for the test data
lm.predict(X_test)

In [ ]:
# let's plot the predicted y values vs the actual y values
import numpy as np
import matplotlib.pyplot as plt
y_pred = lm.predict(X_test)
plt.plot(y_test, y_pred, ".")

x = np.linspace(0, 330, 100)
y = x
plt.plot(x, y)
plt.show()

In [ ]:
# how did we do?
from sklearn.metrics import mean_squared_error, r2_score

print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
print("")
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

### Variants of Linear Regression

We can sometimes get better results by adding a **regularization** or **penalty** term.

### Ridge Regression

In **ridge regression** (or **L2 regularization**), rather than minimizing $\sum_{i=1}^{n} (y_i-\hat{y_i})^2$, we minimize $$\sum_{i=1}^{n} (y_i-\hat{y_i})^2 + \lambda \sum_{i=1}^{n}\beta_{i}^{2}$$ where
*   $\lambda$ is a penalty parameter we choose. The higher the value we choose, the smaller our coefficients end up being.
*   $\beta_i$ are the coefficients selected by the model.

Ridge regression favors smaller coefficients and reduces the overall variance of the model.

### LASSO Regression

In **LASSO regression** (or **L1 regularization**), we minimize $$\sum_{i=1}^{n} (y_i-\hat{y_i})^2 + \lambda \sum_{i=1}^{n} \lvert \beta_{i} \rvert$$

Lasso regression favors smaller coefficients and sometimes even makes them equal 0 (it eliminates variables), which is good if you want a simpler model but bad in that it increases the bias of your model.

## Neural networks

Aside from the regression and classification methods we've already discussed, we can also use **neural networks**! But, we're probably running low on time now, and they're more complicated than the models we've already discussed, so we're saving them for next week.

# Anonymous feedback

If you have any feedback for us, please let us know! The feedback form is completely anonymous, and we promise we'll take your suggestions into account for future presentations: https://forms.gle/C12vK71RJK6CraZv5

# References

Throughout the quarter, we will mainly be drawing our material from the following sources. Most of your learning will be done through trial and error, so we strongly encourage you to experiment by running code that you write from scratch!

For basic Python:
* The Python Tutorial: https://docs.python.org/3/tutorial/
* Basics of Python 3: https://www.learnpython.org/
* CodeAcademy Python 3 Course: https://www.codecademy.com/learn/learn-python-3

For the rest of the quarter:
* Introducing Data Science: http://bedford-computing.co.uk/learning/wp-content/uploads/2016/09/introducing-data-science-machine-learning-python.pdf 
* Python for Data Analysis: http://bedford-computing.co.uk/learning/wp-content/uploads/2015/10/Python-for-Data-Analysis.pdf 
* Pandas user guide: https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html 
* Sklearn user guide: https://scikit-learn.org/stable/user_guide.html 